# Scikit-Learn IRIS Model

 * Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Depenencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install sklearn
pip install grpcio-tools
```

## Train locally
 

In [1]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn import datasets

def main():
    clf = LogisticRegression()
    p = Pipeline([('clf', clf)])
    print('Training model...')
    p.fit(X, y)
    print('Model trained!')

    filename_p = 'IrisClassifier.sav'
    print('Saving model in %s' % filename_p)
    joblib.dump(p, filename_p)
    print('Model saved!')
    
if __name__ == "__main__":
    print('Loading iris data set...')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print('Dataset loaded!')
    main()


Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in IrisClassifier.sav
Model saved!


Wrap model using s2i

## REST test

In [2]:
!s2i build . seldonio/seldon-core-s2i-python3 sklearn-iris:0.1

---> Installing application source...
---> Installing dependencies ...
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [3]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 sklearn-iris:0.1

92a3d2d30811f084881df501b16eba1ac52334927ba407ecf07f627a9c67015b


In [4]:
!cd ../../../wrappers/testing && make build_protos

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


Send some random features that conform to the contract

In [5]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:
{'meta': {}, 'data': {'names': ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], 'ndarray': [[6.369, 3.498, 6.062, 2.57]]}}
RECEIVED RESPONSE:
{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0.00021525713562610643, 0.11504072045280644, 0.8847440224115675]]}}

Time 0.004708290100097656


In [11]:
!docker rm iris_predictor --force

iris_predictor


## grpc test

In [7]:
!s2i build -E .s2i/environment_grpc . seldonio/seldon-core-s2i-python3 sklearn-iris:0.1

---> Installing application source...
---> Installing dependencies ...
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [12]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 sklearn-iris:0.1

b2e8b9153904380d976e3a421d345066fc735d0217efd1e478a0cab70d39ca54


Test using NDArray payload

In [15]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --grpc

----------------------------------------
SENDING NEW REQUEST:
data {
  names: "sepal_length"
  names: "sepal_width"
  names: "petal_length"
  names: "petal_width"
  ndarray {
    values {
      list_value {
        values {
          number_value: 6.893
        }
        values {
          number_value: 4.302
        }
        values {
          number_value: 8.419
        }
        values {
          number_value: 1.102
        }
      }
    }
  }
}

RECEIVED RESPONSE:
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  ndarray {
    values {
      list_value {
        values {
          number_value: 1.3326678349719564e-05
        }
        values {
          number_value: 0.37925859004409174
        }
        values {
          number_value: 0.6207280832775586
        }
      }
    }
  }
}




Test using Tensor payload

In [17]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --grpc --tensor

----------------------------------------
SENDING NEW REQUEST:
data {
  names: "sepal_length"
  names: "sepal_width"
  names: "petal_length"
  names: "petal_width"
  tensor {
    shape: 1
    shape: 4
    values: 7.986
    values: 3.85
    values: 6.16
    values: 0.769
  }
}

RECEIVED RESPONSE:
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  tensor {
    shape: 1
    shape: 3
    values: 0.005626056223331023
    values: 0.9558037301171475
    values: 0.03857021365952149
  }
}




In [ ]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

In [ ]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3 sklearn-iris:0.1

In [ ]:
!kubectl create -f sklearn_iris_deployment.json

Wait until ready (replicas == replicasAvailable)

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' 

In [ ]:
!cd ../../../util/api_tester && make build_protos 

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!minikube delete